## Notebook to research diabetes onset 

In [1]:
%load_ext autoreload
%autoreload 2
#%load_ext Cython
#%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data
from multi_outcomes import multi_outcomes_data
from data_functions import make_date_to_MYR

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
%%time
canary_p = '../../temp/CANARY.p'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
pmpm = pd.read_pickle(pmpm_p)

pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)

CPU times: user 5.96 s, sys: 5.32 s, total: 11.3 s
Wall time: 11.3 s


In [4]:
pmpm.MYR.min(), pmpm.MYR.max(), canary.REGISTER_DATE.min(), canary.REGISTER_DATE.max(), pmpm.shape, canary.shape

(201701,
 202011,
 datetime.date(2017, 10, 28),
 datetime.date(2021, 1, 10),
 (7983305, 74),
 (1526, 12))

### Make canary dates to MYR

In [5]:
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)
canary = canary[['PERS_ID', 'REGISTER_DATE']]
canary['MYR'] = canary.REGISTER_DATE.apply(make_date_to_MYR)
canary['REGISTER_DATE'] = canary['MYR'].values

#### drop canary patients with Register Date  after PMPM maximum date

In [6]:
canary = canary[canary.REGISTER_DATE <= pmpm.MYR.max()].reset_index(drop = True)

### Get controls and Treated

In [7]:
canary_ids = canary.PERS_ID.unique()
treated = pmpm[pmpm.PERS_ID.isin(canary_ids)]
treated = treated.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)
controls = pmpm[~pmpm.PERS_ID.isin(canary_ids)].reset_index(drop = True)

### canary ids not in the pmpm

In [8]:
diff = np.setdiff1d(canary.PERS_ID.unique(), treated.PERS_ID.unique())
canary[canary.PERS_ID.isin(diff)]

PERS_ID  REGISTER_DATE     MYR
114   2740776         202001  202001
142   2758164         201806  201806
164   2768587         201806  201806
397   2887914         201811  201811
726   3077891         202001  202001
830   3222695         201902  201902
1009  3967127         202011  202011
1044  4011417         201912  201912
1275  4310115         202002  202002
1297  4331449         201910  201910
1302  4337577         201912  201912
1327  4355920         201809  201809

### Function to make a flag for when patients were in canary(start from registartion)

In [9]:
def in_canary(group, canary):
    
    idi = group.PERS_ID.values[0]
    
    #a person is considered in canary from its registered date and onwards
    register = canary[canary.PERS_ID == idi].MYR.values[0]
    group.loc[group.MYR >= register, 'IN_CANARY'] = 1
    return group

### Give flag for patients in canary

In [10]:
%%time
treated['IN_CANARY'] = 0
treated2 = treated.groupby('PERS_ID').apply(lambda x: in_canary(x, canary[['PERS_ID', 'MYR']]))

CPU times: user 4.55 s, sys: 21.6 ms, total: 4.58 s
Wall time: 4.57 s


### Get only Relevant data  for Diabetes onset

In [11]:
treated_Diab = treated2[['PERS_ID', 'MYR', 'IN_CANARY', 'CC_DIABETES']]

#keep only patients from their register date and forward
treated_Diab = treated_Diab[treated_Diab['IN_CANARY']>0]

#sort patients by id and myr
treated_Diab = treated_Diab.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)

### Check if for all people in canary in their registered date they did not have diabetes

In [12]:
diabetes_at0 = treated_Diab.groupby('PERS_ID').apply(lambda x: pd.Series({'Diabetes?':x.CC_DIABETES.iloc[0] == 1,
                                                                             'MYR':x.MYR.iloc[0]}))

In [13]:
diabReg = diabetes_at0[diabetes_at0['Diabetes?'] == True]
diabRegID = np.array(diabReg.index)
diabReg.head(5)

Diabetes?     MYR
PERS_ID                   
2608221       True  202010
2619378       True  201910
2680996       True  201903
2686852       True  202003
2721785       True  202005

### find new diabetics

In [14]:
def new_diab(group, whole_dat, months, month_to_index):
    """
    group = month group
    wholde_data = all data including group
    months = a list of months in the data
    month_to_index = a mapping between months and their index
    
    """
    ids = group.PERS_ID.unique()
    myr = group.MYR.values[0]
    ii = month_to_index[myr]
    if ii == 0:
        return
    month_old = months[ii-1]
    dat = whole_dat[whole_dat.MYR == month_old]
    commonids = np.intersect1d(dat.PERS_ID.unique(), ids)
    comp2 = group[group.PERS_ID.isin(commonids)].sort_values(by = ['PERS_ID']).reset_index(drop =True)
    comp1 = dat[dat.PERS_ID.isin(commonids)].sort_values(by = ['PERS_ID']).reset_index(drop = True)
    filter1 = (comp2.CC_DIABETES == 1)& (comp1.CC_DIABETES == 0)
    per_1000 = (10**3/len(group))*(filter1.sum())
    return pd.Series([len(group), filter1.sum(), per_1000], index =['Sample', 'New Diabetic', 'Per 1000'])

    

In [15]:
months = treated_Diab.MYR.sort_values().unique()
months_to_index = dict(zip(months, [i for i in range(len(months))]))

In [16]:
lengths = treated_Diab.groupby('MYR').apply(lambda x: new_diab(x, treated_Diab, months, months_to_index))

### Matching here, get only ids in 201701

In [17]:
match_month = 201710
treatedM = treated2[treated2.MYR == match_month].reset_index(drop = True).copy()
treatedM = treatedM[treatedM.PERS_ID.isin(treated_Diab.PERS_ID.unique())]
treatedM.drop(columns = 'IN_CANARY', inplace = True)

In [18]:
controlsM = controls[controls.MYR == match_month].reset_index(drop = True)


In [19]:
controlsM['TREATED'] = False
treatedM['TREATED'] = True

controlsM.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
treatedM.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)

In [20]:
alli = pd.concat((controlsM, treatedM), axis = 0)
alli.drop(columns = 'CC_DIABETES', inplace = True)
alli['OUTCOME'] = 1

In [21]:
nocols = ['PERS_ID', 'TREATED', 'MYR', 'OUTCOME']
common_causes = np.setdiff1d(alli.columns, nocols)


In [22]:
import dowhy
from dowhy import CausalModel

import logging
logging.getLogger("dowhy").setLevel(logging.WARNING)
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [23]:
data1 = {'df':alli, 
        'common_causes_names':list(common_causes), 
        'outcome':'OUTCOME', 
        'treatment_name':'TREATED'}

In [24]:
def causal_analysis(data, outcome_name,  method = "backdoor.propensity_score_matching"):
    
    #define causal model
    model = CausalModel(
            data=data['df'],
            treatment=data['treatment_name'],
            outcome=data[outcome_name],
            common_causes=data['common_causes_names'],
            )
    
    #check if it is possible the estimand to be estimated
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    
    #calculate propensities
    propensity_match_estimate = model.estimate_effect(identified_estimand,
                            method_name = method,
                            test_significance = False,
                            confidence_intervals = False,
                            target_units="att")
    
    return propensity_match_estimate

In [25]:
m1 = causal_analysis(data1, 'outcome')


/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [26]:
alli.head(3)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  \
0  2570268               93        261        0             1                  1       0          1                    0             1          1              1          0           0       0                 0            0                  0                  1                     0        1                   0              0                        0                  0                0         11                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0            0   
1  2572739               91         94        0             0                  0       0          0                    0             1          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               1             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0            0   
2  2573353               91        182        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  1                     1        0                   0              0                        0                  0                0          6                      0            0                0            0                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           1                0              0               0            0   

   CC_PROSTATE_CANCER 

In [27]:
from sklearn.neighbors import NearestNeighbors

In [28]:
Xtreated = alli[alli.TREATED == True]
Xcontrols = alli[alli.TREATED == False].reset_index(drop = True)

neighbors = NearestNeighbors(n_neighbors=1).fit(X = Xcontrols.propensity_score.values.reshape(-1,1))
matched_indices = neighbors.kneighbors(X = Xtreated.propensity_score.values.reshape(-1,1), n_neighbors=10, return_distance = False)
matched = Xcontrols.iloc[matched_indices[:,0]].reset_index(drop = True)

In [29]:
matched_indices.shape

In [30]:
matched2 = pmpm[pmpm.PERS_ID.isin(matched.PERS_ID.unique())]
matched2 = matched2[matched2.MYR >= match_month].reset_index(drop = True)

In [31]:
matched3 = matched2[['PERS_ID', 'MYR', 'CC_DIABETES']].sort_values(by = ['PERS_ID', 'MYR'])
matched3.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)

In [32]:
matched3.CC_DIABETES.sum(), treated_Diab.CC_DIABETES.sum()

(2728, 520)

In [33]:
matched3

PERS_ID     MYR  CC_DIABETES
17617  2568204  201710            0
17616  2568204  201711            0
17609  2568204  201712            0
17626  2568204  201801            0
17623  2568204  201802            0
...        ...     ...          ...
4387   4357799  202007            1
4380   4357799  202008            1
4401   4357799  202009            1
4408   4357799  202010            1
4410   4357799  202011            1

[22581 rows x 3 columns]

In [34]:
lengths2 = matched3.groupby('MYR').apply(lambda x: new_diab(x, matched3, months, months_to_index))

In [35]:
xx = pd.concat((lengths2.rename(columns = {'New Diabetic':'New_Diab1'}), lengths.rename(columns = {'New Diabetic':'New_Diab2'})), axis = 1)
xx

Sample  New_Diab1  Per 1000  Sample  New_Diab2   Per 1000
MYR                                                              
201710     NaN        NaN       NaN     NaN        NaN        NaN
201711   772.0        0.0  0.000000     2.0        0.0   0.000000
201712   753.0        2.0  2.656042     2.0        0.0   0.000000
201801   714.0        2.0  2.801120     2.0        0.0   0.000000
201802   682.0        1.0  1.466276     8.0        0.0   0.000000
201803   666.0        0.0  0.000000    18.0        0.0   0.000000
201804   664.0        1.0  1.506024    23.0        0.0   0.000000
201805   649.0        1.0  1.540832    30.0        0.0   0.000000
201806   636.0        1.0  1.572327    43.0        0.0   0.000000
201807   629.0        1.0  1.589825    51.0        0.0   0.000000
201808   621.0        3.0  4.830918    56.0        0.0   0.000000
201809   610.0        2.0  3.278689    61.0        0.0   0.000000
201810   607.0        2.0  3.294893    70.0        0.0   0.000000
201811   603.0        0.0  0.000000    99.0        1.0  10.101010
201812   587.0        1.0  1.703578   122.0        0.0   0.000000
201901   570.0        1.0  1.754386   178.0        0.0   0.000000
201902   572.0        5.0  8.741259   210.0        0.0   0.000000
201903   574.0        3.0  5.226481   238.0        0.0   0.000000
201904   574.0        0.0  0.000000   267.0        0.0   0.000000
201905   575.0        2.0  3.478261   288.0        1.0   3.472222
201906   573.0        0.0  0.000000   301.0        0.0   0.000000
201907   564.0        0.0  0.000000   307.0        0.0   0.000000
201908   565.0        0.0  0.000000   326.0        1.0   3.067485
201909   562.0        1.0  1.779359   350.0        0.0   0.000000
201910   557.0        0.0  0.000000   389.0        0.0   0.000000
201911   564.0        1.0  1.773050   549.0        1.0   1.821494
201912   555.0        1.0  1.801802   629.0        1.0   1.589825
202001   546.0        0.0  0.000000   791.0        1.0   1.264223
202002   549.0        4.0  7.285974   869.0        1.0   1.150748
202003   545.0        1.0  1.834862   897.0        2.0   2.229654
202004   541.0        1.0  1.848429   913.0        1.0   1.095290
202005   533.0        0.0  0.000000  1005.0        0.0   0.000000
202006   528.0        0.0  0.000000  1051.0        5.0   4.757374
202007   515.0        1.0  1.941748  1079.0        1.0   0.926784
202008   517.0        1.0  1.934236  1085.0        2.0   1.843318
202009   511.0        1.0  1.956947  1093.0        3.0   2.744739
202010   500.0        1.0  2.000000  1138.0        2.0   1.757469
202011   493.0        1.0  2.028398  1248.0        1.0   0.801282

In [36]:
xx.New_Diab1.mean(), xx.New_Diab2.mean()

In [37]:
lengths2['New Diabetic'].sum()

In [38]:
matched3.CC_DIABETES.sum()

2728

In [39]:
treated_Diab.CC_DIABETES.sum()

520

In [40]:
lengths['New Diabetic'].sum()

In [41]:
matched3.PERS_ID.unique().shape, treated_Diab.PERS_ID.unique().shape

In [42]:
pmpm_DiabID = pmpm[pmpm.CC_DIABETES == 1].PERS_ID.unique()
pmpm_Diab = pmpm[pmpm.PERS_ID.isin(pmpm_DiabID)]

In [43]:
pmpm_Diab = pmpm_Diab[['PERS_ID', 'MYR', 'CC_DIABETES']].sort_values(by = ['PERS_ID', 'MYR'])

In [44]:
def switchesf(group):
    group1 = group.sort_values(by = 'MYR').CC_DIABETES.values
   # print(group1)
    switch01 = 0
    switch10 = 0
    
    for i, val in enumerate(group1):
        #print(val)
        if i == 0:
            continue
       # print('HERE')
        if (val == 1) and (group1[i-1] == 0):
        #    print('HERE0')
            switch01 +=1
        
        if (val == 0) and (group1[i-1] == 1):
           # print('HERE2')
            switch10 += 1
            
        
    return pd.Series([switch01, switch10], index = ['Swithc01', 'Switch10'])
        

In [45]:
%%time
switches = pmpm_Diab.groupby('PERS_ID').apply(switchesf)

CPU times: user 46.8 s, sys: 164 ms, total: 47 s
Wall time: 47 s


In [46]:
switches.sum(axis = 0)

Swithc01    15733
Switch10     6716
dtype: int64

In [47]:
def new_diab2(group, whole_dat, months, month_to_index):
    """
    group = month group
    wholde_data = all data including group
    months = a list of months in the data
    month_to_index = a mapping between months and their index
    
    """
    
    ids = group.PERS_ID.unique()
    myr = group.MYR.values[0]
  #  print(myr)
   # print(group)
    ii = month_to_index[myr]
    if ii == 0:
        return
    month_old = months[ii-1]
    dat = whole_dat[whole_dat.MYR == month_old]
   # print(myr)
    commonids = np.intersect1d(dat.PERS_ID.unique(), ids)
    comp2 = group[group.PERS_ID.isin(commonids)].sort_values(by = ['PERS_ID']).reset_index(drop =True)
   # print(comp2.iloc[0:10])

    comp1 = dat[dat.PERS_ID.isin(commonids)].sort_values(by = ['PERS_ID']).reset_index(drop = True)
   # print(myr,'2')
   # print(comp1.loc[0:10])
    filter1 = (comp2.CC_DIABETES == 1)& (comp1.CC_DIABETES == 0)
   # print(filter1.sum())
    per_1000 = (10**3/len(group))*(filter1.sum())
    return pd.Series([len(group), filter1.sum(), per_1000], index =['Sample', 'New Diabetic', 'Per 1000'])

    

In [48]:
idis = pmpm.PERS_ID.unique()[0:1000]
pmpm2 = pmpm[pmpm.PERS_ID.isin(idis)][['PERS_ID', 'MYR', 'CC_DIABETES']]

In [49]:
%%time
months = np.array(pmpm.MYR.sort_values().unique())
lengths7 = pmpm.groupby('MYR').apply(lambda x: new_diab2(x, pmpm, months, months_to_index))

KeyError: 201701

In [ ]:
lengths7

In [ ]:
lengths7['New Diabetic'].sum()

In [ ]:
one = pmpm2[pmpm2.MYR == 202004]
two = pmpm2[pmpm2.MYR == 202005]

In [ ]:
common = np.intersect1d(one.PERS_ID.unique(), two.PERS_ID.unique())
one.PERS_ID.unique().shape, two.PERS_ID.unique().shape, common.shape

In [ ]:
one1 = one[one.PERS_ID.isin(common)].sort_values(by = 'PERS_ID').reset_index(drop = True)
two1 = two[two.PERS_ID.isin(common)].sort_values(by ='PERS_ID').reset_index(drop = True)

In [ ]:
pmpm.drop_duplicates(subset = ('PERS_ID', 'MYR'), inplace = True)